# Prototipo de documento para el almacenado de data de sensores

<p> La idea de este prototipo es generar una clase con los atributos que los sensores capturan. Luego a través de la carga de un archivo de texto configurar instancias de la clase creada y serializarlas en formato JSON para la carga en la base de datos. </p>

### Cargar Librerias

In [1]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import json

### Cargar Dataset y cambio de formato de fecha

In [2]:

sd = pd.read_csv('sensors_data.csv', sep = ',')
import time
from datetime import datetime
#mmm yy yyyy hh:mm AM/PM
#sd['timestamp'] = sd['timestamp'].apply(lambda x: datetime.strptime(x,'%b %d %Y %H:%M %p'))
sd['timestamp'] = sd['timestamp'].apply(lambda x: "ISODate("+'"'+x+'"'+")")
sd.head(10)


,sensorId,timestamp,lat,lon,temperature,humidity,noise,light,rain,windspeed,city
0,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.803966,-16.869861,24,0.90,-20,22,136,12.93,Quito
1,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.282788,-39.108586,18,0.65,-29,9,623,4.34,Quito
2,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.110735,-22.364947,18,0.20,-44,3,419,24.31,Quito
3,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.392861,-15.974240,16,0.18,-40,32,455,53.03,Quito
4,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.174959,-19.337200,25,0.60,-20,27,533,12.55,Quito
5,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.970454,-9.035936,18,0.62,-17,9,321,0.63,Quito
6,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.553525,-9.774164,22,0.65,-50,18,460,42.82,Quito
7,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.847217,-36.775809,16,0.97,-41,11,669,16.25,Quito
8,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.187356,-9.746359,24,0.79,-14,30,251,2.42,Quito
9,env1982000,"ISODate(""2019-04-28T04:35:23.000Z"")",0.397628,-4.425118,16,0.55,-18,27,177,5.62,Quito


### Verificación de datos numéricos

In [3]:
sd.describe()

,lat,lon,temperature,humidity,noise,light,rain,windspeed
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.506730,-17.913246,18.410000,0.516133,-25.946667,15.753333,389.046667,17.981500
std,0.275817,17.105753,4.047614,0.291014,15.048322,9.688760,174.234507,16.180031
min,0.005446,-71.805654,12.000000,0.010000,-50.000000,0.000000,101.000000,0.000000
25%,0.273601,-24.720912,15.000000,0.290000,-40.000000,8.000000,238.500000,5.367500
50%,0.507424,-12.814444,18.000000,0.510000,-26.000000,15.000000,382.000000,12.845000
75%,0.734839,-4.671502,22.000000,0.770000,-13.000000,24.000000,537.500000,26.347500
max,0.997680,-0.006091,25.000000,0.990000,0.000000,32.000000,697.000000,74.880000


### Listado de Columnas

In [61]:
sd.columns

Index(['sensorId', 'timestamp', 'lat', 'lon', 'temperature', 'humidity',
       'noise', 'light', 'rain', 'windspeed', 'city'],
      dtype='object')

### Verificación de Tipos

In [4]:
sd.dtypes

sensorId        object
timestamp       object
lat            float64
lon            float64
temperature      int64
humidity       float64
noise            int64
light            int64
rain             int64
windspeed      float64
city            object
dtype: object

### Crear archivo JSON

<p>Con la ayuda del codigo generado por Geoff Boeing y que se puede encontrar <a href="https://github.com/gboeing/urban-data-science/blob/master/17-Leaflet-Web-Mapping/leaflet-simple-demo/pandas-to-geojson.ipynb">aquí.</a>
Con unos pequeños ajustes se pudo generar el archivo georeferenciado de prototipo</p>

In [5]:
#sd.to_json('sensors_data.json',orient='records',lines=True)
def df_to_geojson(df, properties, lat='lat', lon='lon'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [6]:

useful_columns = ['sensorId','timestamp', 'lat', 'lon', 'temperature', 'humidity',
       'noise', 'light', 'rain', 'windspeed', 'city']
geojson_dict = df_to_geojson(sd, properties=useful_columns)
geojson_str = json.dumps(geojson_dict, indent=1)

In [7]:

# save the geojson result to a file
output_filename = 'sensor_data.json'
with open(output_filename, 'w') as output_file:
    output_file.write('{};'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))

300 geotagged features saved to file
